# GOOGL: Analyst-Verified Model Inputs (Yahoo Finance via API)


In [ ]:

import sys, subprocess, importlib

def ensure(pkg):
    try:
        importlib.import_module(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

ensure("yfinance")
ensure("pandas")
ensure("numpy")

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta



## 1) Pull core Yahoo Finance fields (price, beta, shares outstanding)
- **Current Stock Price:** Yahoo Finance quote
- **Beta:** Yahoo Finance summary statistics
- **Shares Outstanding:** Yahoo Finance key statistics / share data


In [2]:

ticker = "GOOGL"
t = yf.Ticker(ticker)

# Fast quote fields (more reliable for price than `info` on some days)
fq = getattr(t, "fast_info", None)
price = None
if fq is not None:
    # yfinance fast_info keys can vary; try several
    for k in ["last_price", "lastPrice", "regularMarketPrice"]:
        if k in fq and fq[k] is not None:
            price = float(fq[k])
            break

# Fallback: use 1d history close
if price is None:
    hist_1d = t.history(period="1d")
    price = float(hist_1d["Close"].iloc[-1])

info = {}
try:
    info = t.get_info()
except Exception as e:
    print("Warning: could not load full `info` (Yahoo sometimes throttles). Error:", e)

beta = info.get("beta", np.nan)
shares_out = info.get("sharesOutstanding", np.nan)

print("Ticker:", ticker)
print("Current Stock Price (USD):", price)
print("Beta:", beta)
print("Shares Outstanding:", shares_out)


Ticker: GOOGL
Current Stock Price (USD): 305.7200012207031
Beta: 1.086
Shares Outstanding: 5822000000


## 2) Market capitalization
**Market Cap = Shares Outstanding × Price**


In [3]:

market_cap_calc = np.nan
if pd.notna(shares_out) and pd.notna(price):
    market_cap_calc = float(shares_out) * float(price)

market_cap_yahoo = info.get("marketCap", np.nan)

print("Market Cap (calculated):", market_cap_calc)
print("Market Cap (Yahoo field, if available):", market_cap_yahoo)


Market Cap (calculated): 1779901847106.9336
Market Cap (Yahoo field, if available): 3698295046144


## 3) Balance sheet: Cash & Equivalents, Total Debt, Net Debt
Pulled from Yahoo Finance annual balance sheet (most recent column). If a direct line item is missing, we compute totals from available components.


In [ ]:

bs = t.balance_sheet
display(bs.head(20))

if bs.shape[1] > 0:
    latest_col = bs.columns[0]
else:
    latest_col = None

def get_line(df, names):
    for name in names:
        if name in df.index:
            return df.loc[name]
    return None

cash = np.nan
total_debt = np.nan

if latest_col is not None:
    cash_series = get_line(bs, [
        "Cash And Cash Equivalents",
        "Cash And Cash Equivalents And Short Term Investments",
        "Cash Cash Equivalents And Short Term Investments",
        "Cash And Short Term Investments",
        "Cash"
    ])
    if cash_series is not None:
        cash = float(cash_series[latest_col])

    debt_series = get_line(bs, ["Total Debt"])
    if debt_series is not None:
        total_debt = float(debt_series[latest_col])
    else:
        short_debt = get_line(bs, ["Short Long Term Debt", "Short Term Debt"])
        long_debt  = get_line(bs, ["Long Term Debt", "Long Term Debt And Capital Lease Obligation"])
        vals = []
        if short_debt is not None: vals.append(float(short_debt[latest_col]))
        if long_debt is not None:  vals.append(float(long_debt[latest_col]))
        if vals:
            total_debt = float(np.nansum(vals))

net_debt = np.nan
if pd.notna(total_debt) and pd.notna(cash):
    net_debt = float(total_debt - cash)

print("Latest fiscal column:", latest_col)
print("Cash & Equivalents:", cash)
print("Total Debt:", total_debt)
print("Net Debt (Debt - Cash):", net_debt)


,2025-12-31,2024-12-31,2023-12-31,2022-12-31,2021-12-31
Treasury Shares Number,NaN,NaN,0.0,NaN,NaN
Ordinary Shares Number,12088000000.0,12211000000.0,12460000000.0,12849000000.0,NaN
Share Issued,12088000000.0,12211000000.0,12460000000.0,12849000000.0,NaN
Net Debt,15839000000.0,NaN,NaN,NaN,NaN
Total Debt,59291000000.0,22574000000.0,27121000000.0,29679000000.0,NaN
Tangible Book Value,381885000000.0,293199000000.0,254181000000.0,227184000000.0,NaN
Invested Capital,461812000000.0,335967000000.0,295249000000.0,269001000000.0,NaN
Working Capital,103293000000.0,74589000000.0,89716000000.0,95495000000.0,NaN
Net Tangible Assets,381885000000.0,293199000000.0,254181000000.0,227184000000.0,NaN
Capital Lease Obligations,12744000000.0,11691000000.0,15251000000.0,16822000000.0,NaN


Latest fiscal column: 2025-12-31 00:00:00
Cash & Equivalents: 30708000000.0
Total Debt: 59291000000.0
Net Debt (Debt - Cash): 28583000000.0


## 4) Free Cash Flow (FCF) — most recent year
We compute FCF from Yahoo Finance annual cash flow statement:
**FCF = Operating Cash Flow − Capital Expenditures**
(If Yahoo provides a direct 'Free Cash Flow' line, we also show it.)


In [ ]:

cf = t.cashflow
display(cf.head(30))

if cf.shape[1] > 0:
    cf_col = cf.columns[0]
else:
    cf_col = None

def get_cf_line(df, names):
    for name in names:
        if name in df.index:
            return df.loc[name]
    return None

fcf_direct = np.nan
if cf_col is not None:
    fcf_line = get_cf_line(cf, ["Free Cash Flow"])
    if fcf_line is not None:
        fcf_direct = float(fcf_line[cf_col])

    ocf_line = get_cf_line(cf, ["Total Cash From Operating Activities", "Operating Cash Flow"])
    capex_line = get_cf_line(cf, ["Capital Expenditures", "Capital Expenditure"])
    ocf = float(ocf_line[cf_col]) if ocf_line is not None else np.nan
    capex = float(capex_line[cf_col]) if capex_line is not None else np.nan

    fcf_calc = np.nan
    if pd.notna(ocf) and pd.notna(capex):
        fcf_calc = float(ocf - capex)

print("Latest fiscal column:", cf_col)
print("FCF (direct Yahoo line, if available):", fcf_direct)
print("FCF (calculated OCF - CapEx):", fcf_calc)


,2025-12-31,2024-12-31,2023-12-31,2022-12-31,2021-12-31
Free Cash Flow,73266000000.0,72764000000.0,69495000000.0,60010000000.0,NaN
Repurchase Of Capital Stock,-45709000000.0,-62222000000.0,-61504000000.0,-59296000000.0,NaN
Repayment Of Debt,-32427000000.0,-12701000000.0,-11550000000.0,-54068000000.0,NaN
Issuance Of Debt,64564000000.0,13589000000.0,10790000000.0,52872000000.0,NaN
Capital Expenditure,-91447000000.0,-52535000000.0,-32251000000.0,-31485000000.0,NaN
Income Tax Paid Supplemental Data,NaN,NaN,19164000000.0,18892000000.0,13412000000.0
End Cash Position,30708000000.0,23466000000.0,24048000000.0,21879000000.0,NaN
Beginning Cash Position,23466000000.0,24048000000.0,21879000000.0,20945000000.0,NaN
Effect Of Exchange Rate Changes,208000000.0,-612000000.0,-421000000.0,-506000000.0,NaN
Changes In Cash,7034000000.0,30000000.0,2590000000.0,1440000000.0,NaN


Latest fiscal column: 2025-12-31 00:00:00
FCF (direct Yahoo line, if available): 73266000000.0
FCF (calculated OCF - CapEx): 256160000000.0


## 5) Risk-free rate (Rf): 10-year U.S. Treasury yield
Pulled from Yahoo Finance ticker **^TNX**.



In [ ]:

tnx = yf.Ticker("^TNX")
tnx_hist = tnx.history(period="5d")
tnx_last = float(tnx_hist["Close"].dropna().iloc[-1])

rf_decimal = tnx_last / 1000.0  

print("Latest ^TNX close:", tnx_last)
print("Risk-free rate (decimal):", rf_decimal)
print("Risk-free rate (%):", rf_decimal * 100)


Latest ^TNX close: 4.056000232696533
Risk-free rate (decimal): 0.004056000232696534
Risk-free rate (%): 0.4056000232696534


## 6) Expected market return E(Rm)
Estimated from Yahoo Finance historical prices for **^GSPC** (S&P 500 index) as a proxy.

In [7]:

years = 10

gspc = yf.Ticker("^GSPC")
end = datetime.utcnow().date()
start = end - timedelta(days=int(years * 365.25) + 10)

px = gspc.history(start=start.isoformat(), end=end.isoformat())["Close"].dropna()
start_px = float(px.iloc[0])
end_px = float(px.iloc[-1])

day_count = (px.index[-1].date() - px.index[0].date()).days
year_frac = day_count / 365.25

cagr = (end_px / start_px) ** (1 / year_frac) - 1

print(f"S&P 500 proxy period: {px.index[0].date()} -> {px.index[-1].date()} (~{year_frac:.2f} years)")
print("Start Close:", start_px)
print("End Close:", end_px)
print("Estimated E(Rm) (CAGR, decimal):", cagr)
print("Estimated E(Rm) (%):", cagr * 100)


S&P 500 proxy period: 2016-02-08 -> 2026-02-13 (~10.02 years)
Start Close: 1853.43994140625
End Close: 6836.169921875
Estimated E(Rm) (CAGR, decimal): 0.1391953577654994
Estimated E(Rm) (%): 13.91953577654994


## 7) Summary table
All values below are pulled from Yahoo Finance via the `yfinance` API.

In [10]:

as_of = datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")

summary = pd.DataFrame([
    ["Current Stock Price (USD)", price, "Yahoo Finance quote (yfinance: fast_info / 1d close)", as_of],
    ["Beta", beta, "Yahoo Finance summary stats (yfinance: get_info()['beta'])", as_of],
    ["Shares Outstanding", shares_out, "Yahoo Finance share data (yfinance: get_info()['sharesOutstanding'])", as_of],
    ["Market Cap (calc)", market_cap_calc, "Shares Outstanding × Price (computed)", as_of],
    ["Cash & Equivalents", cash, f"Yahoo Finance balance sheet (yfinance: balance_sheet[{latest_col}] cash line)", as_of],
    ["Total Debt", total_debt, f"Yahoo Finance balance sheet (yfinance: balance_sheet[{latest_col}] debt line)", as_of],
    ["Net Debt", net_debt, "Total Debt − Cash (computed)", as_of],
    ["Free Cash Flow (calc)", fcf_calc, f"Yahoo Finance cash flow (yfinance: cashflow[{cf_col}] OCF - CapEx)", as_of],
    ["Risk-Free Rate (10Y, decimal)", rf_decimal, "Yahoo Finance ^TNX (10Y yield×10) converted: ^TNX/1000", as_of],
    ["Expected Market Return E(Rm) (proxy, decimal)", cagr, f"Yahoo Finance ^GSPC {years}Y CAGR proxy (computed from close prices)", as_of],
], columns=["Metric", "Value", "Source", "As Of"])

summary

,Metric,Value,Source,As Of
0,Current Stock Price (USD),3.057200e+02,Yahoo Finance quote (yfinance: fast_info / 1d ...,2026-02-17 08:03 UTC
1,Beta,1.086000e+00,Yahoo Finance summary stats (yfinance: get_inf...,2026-02-17 08:03 UTC
2,Shares Outstanding,5.822000e+09,Yahoo Finance share data (yfinance: get_info()...,2026-02-17 08:03 UTC
3,Market Cap (calc),1.779902e+12,Shares Outstanding × Price (computed),2026-02-17 08:03 UTC
4,Cash & Equivalents,3.070800e+10,Yahoo Finance balance sheet (yfinance: balance...,2026-02-17 08:03 UTC
5,Total Debt,5.929100e+10,Yahoo Finance balance sheet (yfinance: balance...,2026-02-17 08:03 UTC
6,Net Debt,2.858300e+10,Total Debt − Cash (computed),2026-02-17 08:03 UTC
7,Free Cash Flow (calc),2.561600e+11,Yahoo Finance cash flow (yfinance: cashflow[20...,2026-02-17 08:03 UTC
8,"Risk-Free Rate (10Y, decimal)",4.056000e-03,Yahoo Finance ^TNX (10Y yield×10) converted: ^...,2026-02-17 08:03 UTC
9,"Expected Market Return E(Rm) (proxy, decimal)",1.391954e-01,Yahoo Finance ^GSPC 10Y CAGR proxy (computed f...,2026-02-17 08:03 UTC


## Market Value of Debt

In [11]:
["Market Value of Debt (approx)", total_debt, "Approximation: Market Value of Debt ≈ Total Debt (Yahoo Finance balance sheet)", as_of],

(['Market Value of Debt (approx)',
  59291000000.0,
  'Approximation: Market Value of Debt ≈ Total Debt (Yahoo Finance balance sheet)',
  '2026-02-17 08:03 UTC'],)

## Cost of Debt

In [12]:
interest_expense = np.nan

is_df = t.financials  
display(is_df.head(25))

if is_df.shape[1] > 0:
    is_col = is_df.columns[0]
    for label in ["Interest Expense", "Interest Expense Non Operating"]:
        if label in is_df.index:
            interest_expense = float(is_df.loc[label, is_col])
            break

rd = np.nan
if pd.notna(interest_expense) and pd.notna(total_debt) and total_debt != 0:
    rd = abs(interest_expense) / total_debt

print("Interest Expense:", interest_expense)
print("Cost of Debt Rd (approx):", rd)
print("Cost of Debt Rd (%):", rd * 100)

,2025-12-31,2024-12-31,2023-12-31,2022-12-31,2021-12-31
Tax Effect Of Unusual Items,4119192000.0,340136000.0,-373771000.0,-1035090000.0,NaN
Tax Rate For Calcs,0.168,0.164,0.139,0.159,NaN
Normalized EBITDA,156179000000.0,133320000000.0,100660000000.0,91670000000.0,NaN
Total Unusual Items,24519000000.0,2074000000.0,-2689000000.0,-6510000000.0,NaN
Total Unusual Items Excluding Goodwill,24519000000.0,2074000000.0,-2689000000.0,-6510000000.0,NaN
Net Income From Continuing Operation Net Minority Interest,132170000000.0,100118000000.0,73795000000.0,59972000000.0,NaN
Reconciled Depreciation,21136000000.0,15311000000.0,11946000000.0,13475000000.0,NaN
Reconciled Cost Of Revenue,162535000000.0,146306000000.0,133332000000.0,126203000000.0,NaN
EBITDA,180698000000.0,135394000000.0,97971000000.0,85160000000.0,NaN
EBIT,159562000000.0,120083000000.0,86025000000.0,71685000000.0,NaN


Interest Expense: 736000000.0
Cost of Debt Rd (approx): 0.012413351098817697
Cost of Debt Rd (%): 1.2413351098817698


## Corp Tax Rate

In [13]:
tax_expense = np.nan
pretax_income = np.nan

is_df = t.financials
if is_df.shape[1] > 0:
    is_col = is_df.columns[0]
    for label in ["Pretax Income", "Income Before Tax"]:
        if label in is_df.index:
            pretax_income = float(is_df.loc[label, is_col])
            break
    for label in ["Tax Provision", "Income Tax Expense"]:
        if label in is_df.index:
            tax_expense = float(is_df.loc[label, is_col])
            break

tc = np.nan
if pd.notna(tax_expense) and pd.notna(pretax_income) and pretax_income != 0:
    tc = tax_expense / pretax_income

print("Pretax Income:", pretax_income)
print("Income Tax Expense:", tax_expense)
print("Corporate Tax Rate Tc (effective):", tc)
print("Corporate Tax Rate Tc (%):", tc * 100)

Pretax Income: 158826000000.0
Income Tax Expense: 26656000000.0
Corporate Tax Rate Tc (effective): 0.16783146336242177
Corporate Tax Rate Tc (%): 16.783146336242176
